# Context:
Businesses like banks which provide service have to worry about problem of 'Churn' i.e. customers leaving and joining another service provider. It is important to understand which aspects of the service
influence a customer's decision in this regard. Management can concentrate efforts on improvement of service, keeping in mind these priorities.

# Objective
Given a Bank customer, build a neural network based classifier that can determine whether they will leave or not in the next 6 months.

### Import libraries

In [1]:
!pip install tensorflow==2.0

     |████████████████████████████████| 102.7MB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 11.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 16.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 10.1MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.8MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 11.3MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 450kB 13.6MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 13.2MB/s ta 0:00:01
     |████████████████████████████████| 122kB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 307kB 11.0MB/s eta 0:00:01
     |████████████████████████████████| 163kB 12.0MB/s eta 0:00:01

In [3]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers

### Read Dataset

In [5]:
df = pd.read_csv('bank.csv')

In [6]:
df.shape

(10000, 14)

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [9]:
print("Null in any column? ", df.isnull().values.any())

Null in any column?  False


In [10]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


### Drop Unique Columns

In [17]:
dfn = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [18]:
dfn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### One Hot Encoding

In [19]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [20]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [21]:
dfn = pd.get_dummies(dfn, columns = ['Geography', 'Gender'], drop_first = True)

In [68]:
dfn

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1


In [69]:
dfn['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1


### Feature and Target Variable Selection

In [24]:
features = [col for col in dfn.columns if col != 'Exited']

In [25]:
features

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Geography_Germany',
 'Geography_Spain',
 'Gender_Male']

### Divide into Training and Test Dataset

In [35]:
X_train, X_test, y_train, y_test = train_test_split(dfn[features], dfn['Exited'], test_size = 0.2, random_state = 7)

### Normalize Train and Test Data

In [36]:
X_train = preprocessing.normalize(X_train)

In [37]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 11)
(2000, 11)
(8000,)
(2000,)


In [38]:
X_train

array([[9.83889833e-01, 6.25685109e-02, 1.40779149e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.56421277e-03],
       [4.07481381e-03, 2.14463885e-04, 1.42975923e-05, ...,
        0.00000000e+00, 0.00000000e+00, 7.14879616e-06],
       [3.25738767e-03, 2.69496102e-04, 1.75758327e-05, ...,
        0.00000000e+00, 5.85861092e-06, 5.85861092e-06],
       ...,
       [4.23638988e-03, 2.16774336e-04, 3.71613148e-05, ...,
        0.00000000e+00, 0.00000000e+00, 6.19355246e-06],
       [5.37737031e-03, 2.66516896e-04, 7.05485901e-05, ...,
        0.00000000e+00, 7.83873224e-06, 7.83873224e-06],
       [4.36238487e-03, 1.56444147e-04, 3.61024955e-05, ...,
        0.00000000e+00, 0.00000000e+00, 6.01708258e-06]])

In [39]:
X_test = preprocessing.normalize(X_test)

In [40]:
X_test

array([[4.10249687e-03, 2.00261353e-04, 5.72175295e-06, ...,
        0.00000000e+00, 5.72175295e-06, 5.72175295e-06],
       [4.40738138e-03, 4.68731846e-04, 5.20813162e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.73614534e-03, 2.82097457e-04, 4.27420390e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.34275048e-03, 1.94524530e-04, 4.09525327e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.69067938e-03, 3.36749426e-04, 3.54473080e-05, ...,
        8.86182700e-06, 0.00000000e+00, 8.86182700e-06],
       [3.26165148e-03, 1.76783278e-04, 2.65174917e-05, ...,
        4.41958195e-06, 0.00000000e+00, 0.00000000e+00]])

### Initialize and Build the Model

In [41]:
model = Sequential()

In [42]:
X_train.shape

(8000, 11)

In [75]:
model.add(Dense(64, input_shape = (11,), activation = 'relu'))
model.add(Dense(32, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

### Model compile [optimizers and loss functions]

Keras model should be "compiled" prior to training

Types of loss (function) and optimizer should be designated


In [76]:
sgd = optimizers.Adam(lr = 0.001)
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

### Summary of the model

In [77]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
_________________________________________________________________
dense_3 (Dense)              (None, 64)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
_________________________________________________________________
dense_6 (Dense)              (None, 64)                1

### Training [Forward pass and Backpropagation]

Training the model

In [78]:
model.fit(X_train, y_train.values, batch_size = 700, epochs = 10, verbose = 1)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 112us/sample - loss: 0.6260 - accuracy: 0.7968
Epoch 2/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5447 - accuracy: 0.7968
Epoch 3/10
8000/8000 [==============================] - 0s 5us/sample - loss: 0.5077 - accuracy: 0.7968
Epoch 4/10
8000/8000 [==============================] - 0s 5us/sample - loss: 0.5070 - accuracy: 0.7968
Epoch 5/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5050 - accuracy: 0.7968
Epoch 6/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5051 - accuracy: 0.7968
Epoch 7/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5051 - accuracy: 0.7968
Epoch 8/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5050 - accuracy: 0.7968
Epoch 9/10
8000/8000 [==============================] - 0s 6us/sample - loss: 0.5049 - accuracy: 0.7968
Epoch 10/10
8000/8000 [=================

### Evaluation
Keras model can be evaluated with evaluate() function

Evaluation results are contained in a list



In [79]:
results = model.evaluate(X_test, y_test.values)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================